In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from uszipcode import SearchEngine
from IPython.display import Image
#from geopy.geocoders import Nominatim
#from geopy.distance import geodesic
import us

# Linear Regression w/ l2 norm (Ridge)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge

In [2]:
county_pop_density = pd.read_excel("./Data/2020_UA_COUNTY.xlsx")#, sheet = "2020_UA_COUNTY")
county_pop_density['STATE_NAME']
def state_to_abbreviation(state):
    if state == 'Texas':
        return 'TX'
    elif state == 'New York':
        return 'NY'
    elif state == 'Colorado':
        return 'CO'
    elif state == 'Washington':
        return 'WA'
    else:
        return None # or whatever you want to return if the input is not a valid state name

county_pop_density["STATE_NAME"] = county_pop_density["STATE_NAME"].apply(state_to_abbreviation)


state_county_df = county_pop_density[["STATE_NAME", "COUNTY_NAME"]]
state_county_df = state_county_df[state_county_df["STATE_NAME"].isin(["TX", "CO", "NY", "WA"])]
state_county_df = state_county_df.reset_index()
state_county_df['COUNTY_NAME'] = state_county_df['COUNTY_NAME'].apply(lambda x: x + ' County')



In [3]:
import datetime

def duplicate_rows_by_month(df, start_month, end_month):
    # Convert start and end months to datetime objects
    start_date = datetime.datetime.strptime(start_month, '%Y-%m')
    end_date = datetime.datetime.strptime(end_month, '%Y-%m')
    
    # Create a list of months between start and end months
    months = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m-%d').tolist()
    
    # Create a new dataframe to store duplicated rows
    duplicated_df = pd.DataFrame()
    
    # Iterate over each row in the original dataframe
    for _, row in df.iterrows():
        # Duplicate the row for each month and append to duplicated_df
        duplicated_rows = pd.DataFrame([row] * len(months) * 2, columns=df.columns)
        duplicated_rows['Month'] = months * 2
        duplicated_rows['Drivetrain Type'] = ['BEV'] * len(months) + ['PHEV'] * len(months)
        duplicated_df = pd.concat([duplicated_df, duplicated_rows], ignore_index=True)
    
    return duplicated_df


index_df = duplicate_rows_by_month(state_county_df, "2017-01", "2021-12").drop(columns = ["index"]).rename(columns = {"STATE_NAME" : "State",
                                                                                                            "COUNTY_NAME" : "County",
                                                                                                             "Month" : 'Registration Date'})


# Import Data
Get data from different sources before combining
* Cleaned up EV data: TX_WA_CO_NY.csv
* Average EV price and new car data over time: Avg_EV_Price.csv
* Census data (pop, household income, zipcode): census.csv 

In [4]:
# Import data
df_reg = pd.read_csv('./Data/TX_WA_CO_NY.csv')
df_ev = pd.read_csv('./Data/Avg_EV_Price.csv')
df_c = pd.read_csv('./Data/Census Data/census.csv')

# Convert dates to datetime dtype
df_reg['Registration Date'] = pd.to_datetime(df_reg['Registration Date'])
df_ev['Month'] = pd.to_datetime(df_ev['Month'], format='%b-%y')


In [5]:
df_reg

,State,ZIP Code,Registration Date,Drivetrain Type,Vehicle Count
0,TX,75001,2019-11-01,PHEV,2
1,TX,75001,2020-01-01,PHEV,9
2,TX,75001,2020-02-01,BEV,4
3,TX,75001,2020-04-01,BEV,6
4,TX,75001,2020-05-01,PHEV,9
...,...,...,...,...,...
264421,NY,14905,2023-01-01,PHEV,9
264422,NY,14905,2023-02-01,BEV,2
264423,NY,14905,2023-02-01,PHEV,3
264424,NY,14905,2023-03-01,BEV,1


# Merge Data
## Aggregate Registration Data by County

In [6]:
#Aggregate by County
# create a SearchEngine object
search = SearchEngine()

# define a function to map zip codes to counties
def zipcode_to_county(zipcode):
    #This county does not get populated for some reason
    if zipcode == 75033:
        return "Collin County"
    
    zipcode_data = search.by_zipcode(zipcode)
    county = zipcode_data.county
    return county

# apply the function to create a new column "County"
df_reg['County'] = df_reg['ZIP Code'].apply(zipcode_to_county)

In [7]:
df_reg.shape

(264426, 6)

In [8]:
nan_rows = df_reg[df_reg.isna().any(axis=1)]

In [9]:
df_reg = df_reg.groupby(["State", "Registration Date", "Drivetrain Type", "County"]).agg('sum').drop(columns = ["ZIP Code"]).reset_index()

start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2021-12-31')
df_reg = df_reg[(df_reg['Registration Date'] >= start_date) & (df_reg['Registration Date'] <= end_date)]

In [10]:
df_reg[df_reg['County'] == ""]
df_reg.shape

(19398, 5)

In [11]:
include_zero_month = True

if include_zero_month == True:
    index_df['Registration Date'] = pd.to_datetime(index_df['Registration Date'])
    df_reg['Registration Date'] = pd.to_datetime(df_reg['Registration Date'])


    df_reg = pd.merge(index_df, df_reg, left_on=['State','County','Registration Date', "Drivetrain Type"], right_on=['State','County','Registration Date', "Drivetrain Type"], how='left')
    df_reg = df_reg.fillna(0)

## EV and New Car Prices

In [12]:
# merge ev data in main df
df_reg_ev = pd.merge(df_reg, df_ev, left_on='Registration Date', right_on='Month', how='left')
df_reg_ev = df_reg_ev.drop(['Month'], axis=1)

# Since we don't have ev price data for earlier dates, set all NaN to price for 2020-01-01
fill_val = {'Average EV Price' : df_ev['Average EV Price'][0], 'New Car Average' : df_ev['New Car Average'][0]}
df_reg_ev = df_reg_ev.fillna(value=fill_val)

In [13]:
# check if any nan values
nan_rows = df_reg_ev[df_reg_ev.isna().any(axis=1)]
nan_rows
#df_reg_ev.shape

,State,County,Registration Date,Drivetrain Type,Vehicle Count,Average EV Price,New Car Average


## Census Data

In [14]:
# merge census data
df_reg_ev_c = pd.merge(df_reg_ev, df_c, left_on=['County', "State"], right_on=['county', 'state'], how='left')

df_reg_ev_c = df_reg_ev_c.drop(['Unnamed: 0', 'county'], axis=1)

# check if any nan values
nan_rows = df_reg_ev_c[df_reg_ev_c.isna().any(axis=1)]

print(nan_rows)

# Extract info of missing census data
print('Missing census data in:')
print('counties = ',nan_rows['County'].unique())
print('states = ',nan_rows['State'].unique())
print('Total num of countires = ',len(nan_rows['County'].unique()))
print('Total entries w/ nan = ', len(nan_rows))
#print('Total entries in df = ', len(df))

# The number of missing data is < 1% of total data, just drop
df_reg_ev_c = df_reg_ev_c.dropna()

df_reg_ev_c[['population', 'household_income']] = df_reg_ev_c[['population', 'household_income']].astype(int)


Empty DataFrame
Columns: [State, County, Registration Date, Drivetrain Type, Vehicle Count, Average EV Price, New Car Average, Unnamed: 0.1, population, household_income, state]
Index: []
Missing census data in:
counties =  []
states =  []
Total num of countires =  0
Total entries w/ nan =  0


In [15]:
print(df_reg_ev_c.dtypes)

State                        object
County                       object
Registration Date    datetime64[ns]
Drivetrain Type              object
Vehicle Count               float64
Average EV Price             object
New Car Average              object
Unnamed: 0.1                  int64
population                    int32
household_income              int32
state                        object
dtype: object


# Urban/Rural Divide
Source: https://www2.census.gov/geo/docs/reference/ua/2020_UA_COUNTY.xlsx 
Website: https://www.census.gov/programs-surveys/geography/guidance/geo-areas/urban-rural.html

In [16]:
county_pop_density = pd.read_excel("./Data/2020_UA_COUNTY.xlsx")#, sheet = "2020_UA_COUNTY")
county_pop_density['STATE_NAME']
def state_to_abbreviation(state):
    if state == 'Texas':
        return 'TX'
    elif state == 'New York':
        return 'NY'
    elif state == 'Colorado':
        return 'CO'
    elif state == 'Washington':
        return 'WA'
    else:
        return None # or whatever you want to return if the input is not a valid state name

county_pop_density["STATE_NAME"] = county_pop_density["STATE_NAME"].apply(state_to_abbreviation)

In [17]:
county_pop_density_df = county_pop_density[["STATE_NAME","COUNTY_NAME", "POPDEN_COU"]]
county_pop_density_df['urban_flag'] = [1 if x > 500 else 0 for x in county_pop_density_df['POPDEN_COU']]
county_pop_density_df['COUNTY_NAME'] = county_pop_density_df['COUNTY_NAME'].apply(lambda x: x + ' County')
county_pop_density_df

C:\Users\markm\AppData\Local\Temp\ipykernel_40636\3497240639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_pop_density_df['urban_flag'] = [1 if x > 500 else 0 for x in county_pop_density_df['POPDEN_COU']]
C:\Users\markm\AppData\Local\Temp\ipykernel_40636\3497240639.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_pop_density_df['COUNTY_NAME'] = county_pop_density_df['COUNTY_NAME'].apply(lambda x: x + ' County')


,STATE_NAME,COUNTY_NAME,POPDEN_COU,urban_flag
0,None,Autauga County,98.922916,0
1,None,Baldwin County,145.781265,0
2,None,Barbour County,28.500467,0
3,None,Bibb County,35.814001,0
4,None,Blount County,91.696680,0
...,...,...,...,...
3229,None,Yabucoa County,551.052867,1
3230,None,Yauco County,504.673516,1
3231,None,St. Croix County,491.862425,0
3232,None,St. John County,197.101643,0


In [18]:
df_gas_elec_prices = pd.read_csv("./Data/electricity_gas_prices_reformatted.csv")

# Define a dictionary to map month names to numerical values
month_dict = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 
              'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 
              'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
urban_rural_dict = {"Urban" : 1.0, "Rural" : 0.0}

# Use the map() method to convert the "month" column to numerical values
df_gas_elec_prices['Month'] = df_gas_elec_prices['Month'].map(month_dict)

df_gas_elec_prices['urban_flag'] = df_gas_elec_prices['Population Type'].map(urban_rural_dict)


# Combine "year" and "month" columns into a new column in the format "YYYY-MM"
df_gas_elec_prices['year_month'] = df_gas_elec_prices['Year'].astype(str) + '-' + df_gas_elec_prices['Month']

# Convert the "year_month" column to a datetime object
df_gas_elec_prices['date'] = pd.to_datetime(df_gas_elec_prices['year_month'])


df_gas_elec_prices = df_gas_elec_prices.drop(columns = ["Unnamed: 0", "Population Type", "Year", 
                                            "Month", "year_month"])

df_gas_elec_prices


,State,Electricity Price,Gas Price,urban_flag,date
0,TX,0.1049,2.139,0.0,2017-01-01
1,TX,0.1049,2.083,0.0,2017-02-01
2,TX,0.1049,2.089,0.0,2017-03-01
3,TX,0.1049,2.195,0.0,2017-04-01
4,TX,0.1049,2.187,0.0,2017-05-01
...,...,...,...,...,...
531,CO,0.1570,3.691,1.0,2022-10-01
532,CO,0.1520,3.429,1.0,2022-11-01
533,CO,0.1520,2.979,1.0,2022-12-01
534,CO,0.1530,3.479,1.0,2023-01-01


In [19]:
df = pd.merge(df_reg_ev_c, county_pop_density_df, left_on=['State','County'], right_on=['STATE_NAME','COUNTY_NAME'], how='left')
df = pd.merge(df, df_gas_elec_prices, left_on = ["State", "urban_flag" ,"Registration Date"], right_on = ["State", "urban_flag", "date"], how = "left")
df = df.drop(columns = ["Unnamed: 0.1", "STATE_NAME", "COUNTY_NAME", "date", "urban_flag"])
nan_rows = df[df.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [State, County, Registration Date, Drivetrain Type, Vehicle Count, Average EV Price, New Car Average, population, household_income, state, POPDEN_COU, Electricity Price, Gas Price]
Index: []


In [20]:
df

,State,County,Registration Date,Drivetrain Type,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price
0,CO,Adams County,2017-01-01,BEV,24.0,"$54,669","$38,747",509844,167290,CO,445.323695,0.1266,3.378
1,CO,Adams County,2017-02-01,BEV,25.0,"$54,669","$38,747",509844,167290,CO,445.323695,0.1266,3.378
2,CO,Adams County,2017-03-01,BEV,13.0,"$54,669","$38,747",509844,167290,CO,445.323695,0.1266,3.378
3,CO,Adams County,2017-04-01,BEV,15.0,"$54,669","$38,747",509844,167290,CO,445.323695,0.1266,3.378
4,CO,Adams County,2017-05-01,BEV,21.0,"$54,669","$38,747",509844,167290,CO,445.323695,0.1266,3.378
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50275,WA,Yakima County,2021-08-01,PHEV,159.0,"$57,540","$43,418",250649,83765,WA,59.780820,0.0972,3.908
50276,WA,Yakima County,2021-09-01,PHEV,158.0,"$56,312","$45,031",250649,83765,WA,59.780820,0.0972,3.908
50277,WA,Yakima County,2021-10-01,PHEV,158.0,"$55,625","$46,026",250649,83765,WA,59.780820,0.0972,3.908
50278,WA,Yakima County,2021-11-01,PHEV,160.0,"$56,437","$46,329",250649,83765,WA,59.780820,0.0972,3.908


# Export Data

In [21]:
# Desired prediction var
predict_label = 'Vehicle Count'

# Drop zip code since it would increase the number of features by ~4k
# also zip code is highly correlated to population and income
# Drop registration date since we will change to Unix timestamps
drop_col = ['state','County', 'Registration Date']

# Assemble Categorical Variables
cat_var = ['State', 'Drivetrain Type']
for cat in cat_var:
    # Get dummy variables for cat
    dummy_var = df[cat].unique()
    dummy_var = dummy_var[1:]

    # create df w/ one hot cat features
    df_cat = pd.get_dummies(df[cat], drop_first=True)

    # drop original
    df = df.drop([cat], axis=1)

    # concatenate
    df = pd.concat([df, df_cat], axis=1)

# Add Unix timestamp   
df['Unix Time'] = df['Registration Date'].apply(lambda x: x.timestamp())

# Convert $ price into integer
df['Average EV Price'] = df['Average EV Price'].str.replace('$','').str.replace(',','').astype(int)
df['New Car Average'] = df['New Car Average'].str.replace('$','').str.replace(',','').astype(int)

df

C:\Users\markm\AppData\Local\Temp\ipykernel_40636\3669364176.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Average EV Price'] = df['Average EV Price'].str.replace('$','').str.replace(',','').astype(int)
C:\Users\markm\AppData\Local\Temp\ipykernel_40636\3669364176.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['New Car Average'] = df['New Car Average'].str.replace('$','').str.replace(',','').astype(int)


,County,Registration Date,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time
0,Adams County,2017-01-01,24.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.483229e+09
1,Adams County,2017-02-01,25.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.485907e+09
2,Adams County,2017-03-01,13.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.488326e+09
3,Adams County,2017-04-01,15.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.491005e+09
4,Adams County,2017-05-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.493597e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50275,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.627776e+09
50276,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.630454e+09
50277,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.633046e+09
50278,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.635725e+09


# Add Fips and Predict both PHEV and BEV
Currently we have Vehicle Count we are predicting with a one hot vector for PHEV

However, we actually want to predict both of these values so we should have two columns per them

Essentially we have duplicate nodes that we need to get rid of for the GNN

In [39]:
# df = pd.read_csv('./Data/df_all_features_county.csv')
# df

,County,Registration Date,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,Adams County,2017-01-01,24.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.483229e+09,8001
1,Adams County,2017-02-01,25.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.485907e+09,8001
2,Adams County,2017-03-01,13.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.488326e+09,8001
3,Adams County,2017-04-01,15.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.491005e+09,8001
4,Adams County,2017-05-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.493597e+09,8001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50275,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.627776e+09,53077
50276,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.630454e+09,53077
50277,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.633046e+09,53077
50278,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.635725e+09,53077


In [40]:
state_list = df['state'].to_list()
county_list = df['County'].to_list()
county_state_list = [county_list[i] + ', ' + state_list[i] for i in range(len(state_list))]
county_state_list

['Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams County, CO',
 'Adams Count

In [41]:
# read adjacency
df_a = pd.read_csv('./Data/df_adjacency.csv')
df_a

,countyname,fipscounty,neighborname,fipsneighbor,county,state,neighbor_county,neighbor_state,count,distance_(mi),Coordinates,county_lat,county_lon
0,"Adams County, CO",8001,"Adams County, CO",8001,Adams County,CO,Adams County,CO,1,0.000000,"(39.8714085, -104.2701374)",39.871409,-104.270137
1,"Adams County, CO",8001,"Arapahoe County, CO",8005,Adams County,CO,Arapahoe County,CO,1,16.600485,"(39.8714085, -104.2701374)",39.871409,-104.270137
2,"Adams County, CO",8001,"Broomfield County, CO",8014,Adams County,CO,Broomfield County,CO,1,683.581802,"(39.8714085, -104.2701374)",39.871409,-104.270137
3,"Adams County, CO",8001,"Denver County, CO",8031,Adams County,CO,Denver County,CO,1,649.967579,"(39.8714085, -104.2701374)",39.871409,-104.270137
4,"Adams County, CO",8001,"Jefferson County, CO",8059,Adams County,CO,Jefferson County,CO,1,56.499723,"(39.8714085, -104.2701374)",39.871409,-104.270137
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,"Yakima County, WA",53077,"Klickitat County, WA",53039,Yakima County,WA,Klickitat County,WA,1,37.217994,"(46.4135961, -120.730253)",46.413596,-120.730253
2886,"Yakima County, WA",53077,"Lewis County, WA",53041,Yakima County,WA,Lewis County,WA,1,79.821283,"(46.4135961, -120.730253)",46.413596,-120.730253
2887,"Yakima County, WA",53077,"Pierce County, WA",53053,Yakima County,WA,Pierce County,WA,1,81.294253,"(46.4135961, -120.730253)",46.413596,-120.730253
2888,"Yakima County, WA",53077,"Skamania County, WA",53059,Yakima County,WA,Skamania County,WA,1,65.230716,"(46.4135961, -120.730253)",46.413596,-120.730253


In [42]:
# Create a dictionary that maps the 'fruit' column of the DataFrame to the 'color' column
map_dict = dict(zip(df_a['countyname'], df_a['fipscounty']))

# Use the map() function to map the values in the list to the corresponding values in the DataFrame column
mapped_fips = list(map(lambda f: map_dict[f], county_state_list))
df['fips'] = mapped_fips
df.to_csv('./Data/df_all_features_county.csv', index = False)
df

,County,Registration Date,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,Adams County,2017-01-01,24.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.483229e+09,8001
1,Adams County,2017-02-01,25.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.485907e+09,8001
2,Adams County,2017-03-01,13.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.488326e+09,8001
3,Adams County,2017-04-01,15.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.491005e+09,8001
4,Adams County,2017-05-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.493597e+09,8001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50275,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.627776e+09,53077
50276,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.630454e+09,53077
50277,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.633046e+09,53077
50278,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1,1.635725e+09,53077


In [43]:
df_phev = df[df['PHEV'] == 1]
df_phev.drop('PHEV', axis=1, inplace=True)
df_phev

C:\Users\markm\AppData\Local\Temp\ipykernel_40636\2744734323.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phev.drop('PHEV', axis=1, inplace=True)


,County,Registration Date,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips
60,Adams County,2017-01-01,22.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.483229e+09,8001
61,Adams County,2017-02-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.485907e+09,8001
62,Adams County,2017-03-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.488326e+09,8001
63,Adams County,2017-04-01,14.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.491005e+09,8001
64,Adams County,2017-05-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.493597e+09,8001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50275,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.627776e+09,53077
50276,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.630454e+09,53077
50277,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.633046e+09,53077
50278,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.635725e+09,53077


In [44]:
df_bev = df[df['PHEV']==0]
#df_bev = df_bev[['Registration Date','fips','Vehicle Count']]
df_bev

,County,Registration Date,Vehicle Count,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,PHEV,Unix Time,fips
0,Adams County,2017-01-01,24.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.483229e+09,8001
1,Adams County,2017-02-01,25.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.485907e+09,8001
2,Adams County,2017-03-01,13.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.488326e+09,8001
3,Adams County,2017-04-01,15.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.491005e+09,8001
4,Adams County,2017-05-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,0,1.493597e+09,8001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50215,Yakima County,2021-08-01,233.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,0,1.627776e+09,53077
50216,Yakima County,2021-09-01,242.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,0,1.630454e+09,53077
50217,Yakima County,2021-10-01,250.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,0,1.633046e+09,53077
50218,Yakima County,2021-11-01,264.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,0,1.635725e+09,53077


In [45]:
df = pd.merge(df_phev, df_bev[['Registration Date','fips','Vehicle Count']], on=['Registration Date','fips'], how='outer')
df = df.rename(columns={'Vehicle Count_x': 'PHEV', 'Vehicle Count_y': 'BEV'})
df

# merged_df = pd.merge(df_phev, df_bev, on=['Registration Date','fips'], how='outer')
# merged_df 



,County,Registration Date,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,Adams County,2017-01-01,22.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.483229e+09,8001,24.0
1,Adams County,2017-02-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.485907e+09,8001,25.0
2,Adams County,2017-03-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.488326e+09,8001,13.0
3,Adams County,2017-04-01,14.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.491005e+09,8001,15.0
4,Adams County,2017-05-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.493597e+09,8001,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.627776e+09,53077,233.0
25136,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.630454e+09,53077,242.0
25137,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.633046e+09,53077,250.0
25138,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.635725e+09,53077,264.0


In [29]:
# column_names = df_phev.columns.tolist()

In [30]:
# # Select rows from df1 that do not have a match in df2
# df1_unmatched = merged_df[merged_df['state_y'].isna()]
# df1_unmatched = df1_unmatched.dropna(axis=1)
# col_dic = dict(zip(df1_unmatched.columns, column_names))
# df1_unmatched = df1_unmatched.rename(columns=col_dic)
# df1_unmatched.drop('PHEV', axis=1, inplace=True)
# df1_unmatched = df1_unmatched.rename(columns={'Vehicle Count': 'PHEV'})
# df1_unmatched['BEV'] = 0
# df1_unmatched


,County,Registration Date,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,...,state_y,POPDEN_COU_y,Electricity Price_y,Gas Price_y,NY_y,TX_y,WA_y,PHEV_y,Unix Time_y,BEV


In [31]:
# # Select rows from df2 that do not have a match in df1
# df2_unmatched = merged_df[merged_df['state_x'].isna()]
# df2_unmatched = df2_unmatched.dropna(axis=1)
# col_2 = df2_unmatched.pop('fips')
# df2_unmatched = df2_unmatched.assign(fips=col_2)
# col_dic = dict(zip(df2_unmatched.columns, column_names))
# df2_unmatched = df2_unmatched.rename(columns=col_dic)
# df2_unmatched.drop('PHEV', axis=1, inplace=True)

# df2_unmatched = df2_unmatched.rename(columns={'Vehicle Count': 'PHEV'})
# df2_unmatched = df2_unmatched.assign(BEV=df2_unmatched['PHEV'])
# df2_unmatched['PHEV'] = 0

# df2_unmatched

,County,Registration Date,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,...,POPDEN_COU_y,Electricity Price_y,Gas Price_y,NY_y,TX_y,WA_y,PHEV_y,Unix Time_y,fips,BEV


In [32]:
# merged_df = pd.merge(df_phev, df_bev[['Registration Date', 'fips', 'Vehicle Count']], on=['Registration Date','fips'])
# merged_df.drop('PHEV', axis=1, inplace=True)
# merged_df = merged_df.rename(columns={'Vehicle Count_x': 'PHEV', 'Vehicle Count_y': 'BEV'})
# merged_df 

,County,Registration Date,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,Adams County,2017-01-01,22.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.483229e+09,8001,24.0
1,Adams County,2017-02-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.485907e+09,8001,25.0
2,Adams County,2017-03-01,21.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.488326e+09,8001,13.0
3,Adams County,2017-04-01,14.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.491005e+09,8001,15.0
4,Adams County,2017-05-01,27.0,54669,38747,509844,167290,CO,445.323695,0.1266,3.378,0,0,0,1.493597e+09,8001,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,Yakima County,2021-08-01,159.0,57540,43418,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.627776e+09,53077,233.0
25136,Yakima County,2021-09-01,158.0,56312,45031,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.630454e+09,53077,242.0
25137,Yakima County,2021-10-01,158.0,55625,46026,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.633046e+09,53077,250.0
25138,Yakima County,2021-11-01,160.0,56437,46329,250649,83765,WA,59.780820,0.0972,3.908,0,0,1,1.635725e+09,53077,264.0


In [33]:
# df = pd.concat([merged_df, df1_unmatched, df2_unmatched], axis=0)
# df.to_csv('./Data/df_all_features_county.csv', index = False)
# df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

# Export Data

In [47]:
# Desired prediction var
predict_label = ['PHEV', 'BEV']

# Drop zip code since it would increase the number of features by ~4k
# also zip code is highly correlated to population and income
# Drop registration date since we will change to Unix timestamps
drop_col = ['state','County', 'Registration Date', 'fips', 'NY', 'TX', 'WA']

# Get labels of all features
features = [c for c in df.columns if c not in predict_label + drop_col]
features
# extract values to np
y = df[predict_label].to_numpy()
X = df[features].to_numpy()

df = df.reset_index(drop=True)
df.to_csv('./Data/df_all_features_county.csv', index = False)
df[features].to_csv('./Data/df_X_county.csv', index = False)
df['PHEV'].to_csv('./Data/df_y_county.csv', index = False)


In [ ]:
df[features]

,Average EV Price,New Car Average,population,household_income,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time
0,54669.0,38747.0,509844.0,167290.0,445.323695,0.1266,3.378,0.0,0.0,0.0,1.483229e+09
1,54669.0,38747.0,649980.0,241889.0,821.038725,0.1200,2.429,0.0,0.0,0.0,1.483229e+09
2,54669.0,38747.0,324682.0,127365.0,455.351666,0.1266,3.378,0.0,0.0,0.0,1.483229e+09
3,54669.0,38747.0,69444.0,27199.0,2248.011733,0.1200,2.429,0.0,0.0,0.0,1.483229e+09
4,54669.0,38747.0,715878.0,287756.0,4674.337363,0.1200,2.429,0.0,0.0,0.0,1.483229e+09
...,...,...,...,...,...,...,...,...,...,...,...
11084,54669.0,38747.0,13588.0,5798.0,9.570523,0.0953,3.908,0.0,0.0,1.0,1.538352e+09
11085,54669.0,38747.0,11906.0,4866.0,7.258144,0.0953,3.908,0.0,0.0,1.0,1.538352e+09
11086,54669.0,38747.0,22121.0,9714.0,25.026666,0.0953,3.908,0.0,0.0,1.0,1.541030e+09
11087,54669.0,38747.0,19702.0,6035.0,10.708173,0.0952,3.908,0.0,0.0,1.0,1.546301e+09


In [ ]:
df.head()

,Registration Date,County,PHEV,Average EV Price,New Car Average,population,household_income,state,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time,fips,BEV
0,2017-01-01,Adams County,22.0,54669.0,38747.0,509844.0,167290.0,CO,445.323695,0.1266,3.378,0.0,0.0,0.0,1.483229e+09,8001,24.0
1,2017-01-01,Arapahoe County,61.0,54669.0,38747.0,649980.0,241889.0,CO,821.038725,0.1200,2.429,0.0,0.0,0.0,1.483229e+09,8005,65.0
2,2017-01-01,Boulder County,63.0,54669.0,38747.0,324682.0,127365.0,CO,455.351666,0.1266,3.378,0.0,0.0,0.0,1.483229e+09,8013,167.0
3,2017-01-01,Broomfield County,7.0,54669.0,38747.0,69444.0,27199.0,CO,2248.011733,0.1200,2.429,0.0,0.0,0.0,1.483229e+09,8014,14.0
4,2017-01-01,Denver County,56.0,54669.0,38747.0,715878.0,287756.0,CO,4674.337363,0.1200,2.429,0.0,0.0,0.0,1.483229e+09,8031,94.0


In [ ]:
df[predict_label].head()

,PHEV,BEV
0,22.0,24.0
1,61.0,65.0
2,63.0,167.0
3,7.0,14.0
4,56.0,94.0


In [ ]:
df[features].head()

,Average EV Price,New Car Average,population,household_income,POPDEN_COU,Electricity Price,Gas Price,NY,TX,WA,Unix Time
0,54669.0,38747.0,509844.0,167290.0,445.323695,0.1266,3.378,0.0,0.0,0.0,1.483229e+09
1,54669.0,38747.0,649980.0,241889.0,821.038725,0.1200,2.429,0.0,0.0,0.0,1.483229e+09
2,54669.0,38747.0,324682.0,127365.0,455.351666,0.1266,3.378,0.0,0.0,0.0,1.483229e+09
3,54669.0,38747.0,69444.0,27199.0,2248.011733,0.1200,2.429,0.0,0.0,0.0,1.483229e+09
4,54669.0,38747.0,715878.0,287756.0,4674.337363,0.1200,2.429,0.0,0.0,0.0,1.483229e+09


# Adjacency Code

In [ ]:
# state_order = ['CO', 'NY', 'TX', 'WA']
# # drop counties that have nan in state
# df_c.dropna(subset=['state'], inplace=True)
# df_c
# df_c['state'].value_counts()['TX']
# df_a = pd.read_csv('./Data/county_adjacency2010.csv')
# df_a
# df_a[['county','state']] = df_a['countyname'].str.split(', ', expand=True)
# df_a[['neighbor_county', 'neighbor_state']] = df_a['neighborname'].str.split(', ', expand=True)
# df_a = df_a[df_a['state'].isin(state_order)]
# df_a
# # Create a geolocator object
# geolocator = Nominatim(user_agent='my_app')

# # Define a function to calculate distance between two counties
# def calculate_distance(row):
#     # Look up latitude and longitude coordinates for county and neighbor
#     county = geolocator.geocode(row['county'] + ', ' + row['state'] + ', USA')
#     neighbor = geolocator.geocode(row['neighbor_county'] + ', ' + row['neighbor_state'] + ', USA')
#     # Calculate distance using Haversine formula
#     try:
#         distance = geodesic((county.latitude, county.longitude), (neighbor.latitude, neighbor.longitude)).miles
#     except:
#         distance = 0
#     return distance

# # Apply the function to each row of the dataframe
# df_a['distance_(mi)'] = df_a.apply(calculate_distance, axis=1)
# df_a.to_csv('./Data/df_adjacency.csv', index = False)
# df_a
# df_zeros = df_a[df_a['distance_(mi)'] == 0]
# df_zeros.to_csv('./Data/df_zeros.csv', index = False)
# nm_rows = df_a[df_a['neighborname'].str.contains('NM', na=False)]
# len(nm_rows)
# # Create a dataframe that lists all neighboring county pairs
# df_a['count'] = 1
# neighbor_pairs = df_a[['fipscounty', 'fipsneighbor', 'count', 'distance_(mi)']]

# # Create a pivot table with neighboring county names as row and column indices
# adjacency_matrix = df_a.pivot_table(index='fipscounty', columns='fipsneighbor', values='count', fill_value=0)

# # Convert the pivot table to a numpy array to get the adjacency matrix
# distance_matrix = df_a.pivot_table(index='fipscounty', columns='fipsneighbor', values='distance_(mi)', fill_value=0)

KeyboardInterrupt: 